In [1]:
""" Custom batch class for storing mnist batch and models

"""
import sys

import numpy as np
import os
import blosc

import tensorflow as tf
import matplotlib.pyplot as plt

import os
import blosc

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
sys.path.append("..")


from dataset import DatasetIndex, Dataset

from dataset import Batch, action, model, inbatch_parallel, ImagesBatch

ImportError: cannot import name 'ImagesBatch'

In [ ]:
class MnistBatch(ImagesBatch):
    """ Mnist batch and models
    """
    def __init__(self, index, *args, **kwargs):
        """ Init func, inherited from base batch
        """
        super().__init__(index, *args, **kwargs)
        self.images = None
        self.labels = None



    def post_function(self, list_results):
        '''Post function for parallel shift, gathers results of every worker'''
        result_batch = np.array(list_results)
        self.images = result_batch
        return self

    def init_function(self):
        '''Init function for parallel shift
        returns list of indices, each of them will be sent to the worker separately
        '''
        return range(self.images.shape[0])

    @action
    @inbatch_parallel(init='init_function', post='post_function', target='threads')
    def shift_flattened_pic(self, idx, max_margin=8):
        """ Apply random shift to a flattened pic
        
        Args:
            pic: ndarray of shape=(784) representing a pic to be flattened
        Return:
            flattened shifted pic
        """
        
        pic = self.images[idx]
        padded = np.pad(pic, pad_width=[[max_margin, max_margin], [max_margin, max_margin]], 
                        mode='minimum')
        left_lower = np.random.randint(2 * max_margin, size=2)
        slicing = (slice(left_lower[0], left_lower[0] + 28),
                   slice(left_lower[1], left_lower[1] + 28))
        res = padded[slicing]
        return res

    
    @property
    def components(self):
        """ Components of mnist-batch
        """
        return 'images', 'labels'

    @action
    def load(self, src, fmt='blosc'):
        """ Load mnist pics with specifed indices

        Args:
            fmt: format of source. Can be either 'blosc' or 'ndarray'
            src: if fmt='blosc', then src is a path to dir with blosc-packed
                mnist images and labels are stored.
                if fmt='ndarray' - this is a tuple with arrays of images and labels

        Return:
            self
        """
        if fmt == 'blosc':     
            # read blosc images, labels
            with open('mnist_pics.blk', 'rb') as file:
                self.images = blosc.unpack_array(file.read())[self.indices]
                self.images = np.reshape(self.images, (65000, 28, 28))

            with open('mnist_labels.blk', 'rb') as file:
                self.labels = blosc.unpack_array(file.read())[self.indices]
        elif fmt == 'ndarray':
            all_images, all_labels = src
            self.images = all_images[self.indices]
            self.labels = all_labels[self.indices]

        return self

    @model()
    def convy(mode='dynamic'):
        """ Conv-net mnist classifier

        Args:
            ___
        Return:
            [[placeholder for input, ph for true labels, loss, train_step],
             [true categorical labels, categorical_hat labels, accuracy]]
        """
        graph = tf.Graph()
        with graph.as_default():
            # build the net
            training = tf.placeholder(tf.bool, shape=[], name='mode')
            x = tf.placeholder(tf.float32, [None, 28, 28], name='x')
            x_as_pics = tf.reshape(x, shape=[-1, 28, 28, 1])


            net = tf.layers.conv2d(x_as_pics, filters=4, kernel_size=(7,7), strides=(1, 1), padding='same')
            net = tf.layers.max_pooling2d(net, pool_size=(6, 6), strides=(2, 2), padding='same')
            net = tf.layers.batch_normalization(net, training=training)
            net = tf.nn.relu(net)

            net = tf.layers.conv2d(net, filters=16, kernel_size=(5, 5), strides=(1, 1), padding='same')
            net = tf.layers.max_pooling2d(net, pool_size=(5, 5), strides=(2, 2), padding='same')
            net = tf.layers.batch_normalization(net, training=training)
            net = tf.nn.relu(net)


            net = tf.layers.conv2d(net, filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same')
            net = tf.layers.max_pooling2d(net, pool_size=(2, 2), strides=(2, 2), padding='same')
            net = tf.layers.batch_normalization(net, training=training)
            net = tf.nn.relu(net)


            net = tf.contrib.layers.flatten(net)

            # dropout 
            keep_prob = tf.placeholder(tf.float32)
            # net = tf.nn.dropout(net, keep_prob)


            net = tf.layers.dense(net, 128, kernel_initializer=tf.truncated_normal_initializer(0.0, 1))
    #         net = tf.layers.dense(net, 128, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False))

            net = tf.nn.relu(net)

            net = tf.layers.dense(net, 10, kernel_initializer=tf.truncated_normal_initializer(0.0, 1))
    #         net = tf.layers.dense(net, 10, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False))

            probs = tf.nn.softmax(logits=net, name='softmax_output')

            # placeholder for correct labels
            y_ = tf.placeholder(tf.float32, [None, 10], name='y_')

            # loss
    #         loss = tf.nn.softmax_cross_entropy_with_logits(logits=net, labels=y_, name='loss')
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=net, labels=y_, name='loss'))

            global_step = tf.Variable(0, trainable=False)
            starter_learning_rate = 0.0001

            learning_rate = tf.placeholder(tf.float32, shape=[], name='lr')
            # learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
            #                                           100, 0.85, staircase=True)

            # optimization step
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                train_step = (
                    tf.train.GradientDescentOptimizer(learning_rate)
                    .minimize(loss, global_step=global_step)
                    )

            # stats
            labels_hat = tf.cast(tf.argmax(net, axis=1), tf.float32, name='labels_hat')
            labels = tf.cast(tf.argmax(y_, axis=1), tf.float32, name='labels')
            accuracy = tf.reduce_mean(tf.cast(tf.equal(labels_hat, labels), tf.float32), name='accuracy')

            softmax = tf.placeholder(tf.float32, [None, 10], name='softmax')

            predicts = tf.placeholder(tf.float32, [None, 10], name='predicts')
            test_acc = tf.reduce_mean(tf.cast(tf.equal(predicts, labels), tf.float32), name='accuracy')

            print_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=softmax, labels=y_, name='print_loss'))
            
            cyclic_learning_rate = alpha / tf.cast(2.0, tf.float32) * 
                (tf.cos(tf.cast(np.pi, tf.float32) * ((tf.cast(global_step, tf.float32) - 1) % (period)) / (period)) + 1)          

            sess = tf.Session()
            sess.run(tf.global_variables_initializer())

        return [[x, y_, loss, train_step, training, keep_prob], [labels, labels_hat, accuracy], [probs],
                [learning_rate, global_step, print_loss], [softmax, predicts, test_acc], [sess], [cyclic_learning_rate]]

    @action(model='convy')
    def predict(self, model, pics, y_true, y_predict, probabilities):
        ''' Predict labels '''
        sess = model[5][0]
        x, y_, _, _, training, keep_prob = model[0]
        labels, labels_hat, _ = model[1]
        probs = model[2][0]
        probabilities.append(sess.run(probs, feed_dict={x:self.images, training: False, keep_prob: 1.0}))
        y_predict.append(sess.run(labels_hat, feed_dict={x:self.images, training: False, keep_prob: 1.0}))
        y_true.append(sess.run(labels, feed_dict={y_:self.labels}))
        pics.append(self.images)
        return self

    @action(model='convy')
    def train_convy(self, model, alpha, period, n_iterations, accs, loss_history):
        """ Train-action for convy-model

        Args:
            model: do not supply this arg, always the output of convy-model defined above
            sess: tf-session in which learning variables are to be updated
        """        
        sess = model[5][0]
        cyclic_learning_rate = model[6][0]
        x, y_, loss, train_step, training, keep_prob = model[0]
        learning_rate, global_step, _ = model[3]
        
        _, _, accuracy = model[1]

        alpha = tf.cast(alpha, tf.float32)
        
        global_step = sess.run(global_step)
    
        period = tf.cast(period, tf.float32)
        n_iterations = tf.cast(n_iterations, tf.float32)
        n_cycles = n_iterations // period
        
        
        cyclic_learning_rate = sess.run(cyclic_learning_rate)
        # print ('HHHHHHHHEEEEEEEEEEEY', cyclic_learning_rate)
        sess.run(train_step, feed_dict={x: self.images, y_: self.labels, training: True, keep_prob: 0.7, learning_rate:cyclic_learning_rate})        
        
        period = sess.run(period)
        
        if (global_step) % period == 0:
            if global_step == 0:
                pass
            else:
                print ('hey')
                saver = tf.train.Saver(name=str(global_step))
                address = 'trained' + '+' + str(global_step) + '/model'
                saver.save(sess, address, global_step=global_step)
        
#         if global_step == (n_iterations - 1):
#             print ('hey')
#             saver = tf.train.Saver()
#             saver.save(sess, 'trained/model', global_step=global_step)

        
        loss_history.append(sess.run(loss, feed_dict={x: self.images, y_: self.labels, training: False, keep_prob: 1.0}))

        accs.append(sess.run(accuracy, feed_dict={x: self.images, y_: self.labels, training: False, keep_prob: 1.0}))

        return self

    
#     @model()
#     def ensemble():
#         ''' Classifier which averages prediction from m models loaded from the disk 
#             Args:
#             __
#             Returns:
#         '''
#         n_cycles = n_iterations // period
#         results = []
#         for i in range(1, n_cycles + 1):
#             address = 'trained/model' + '-' + str(i*period) + '.meta'

#             grapphy_2 = tf.Graph()
#             with grapphy_2.as_default():
#                 new_sess = tf.Session()

#                 new_saver = tf.train.import_meta_graph(address)
#                 new_saver.restore(new_sess, tf.train.latest_checkpoint('trained/'))
#                 training = grapphy_2.get_tensor_by_name('mode:0')
#                 x = grapphy_2.get_tensor_by_name('x:0')
#                 softmax_output = grapphy_2.get_tensor_by_name('softmax_output:0')

#                 res = new_sess.run(softmax_output, feed_dict={x:imgs, training:False})
#                 results.append(res)
                
#         return results
            

    
#     @action(model='ensemble')
#     def update_stats(self, model):
#         results = model
        
    
    @action(model='convy')
    def update_stats(self, model, period, n_iterations, accs, loss_history):
        """ Append accuracy that is obtained by convy-model given weights stored in sess Tf-session

        Args:
            model: do not supply this arg, always the output of convy-model defined above
            sess: tf-session with trained (to some extent) weights
            accs: list with accuracies
        """
        sess = model[5][0]
        n_cycles = n_iterations // period
        results = []
        for i in range(1, n_cycles + 1):
            address = 'trained/model' + '-' + str(i*period) + '.meta'

            grapphy_2 = tf.Graph()
            with grapphy_2.as_default():
                new_sess = tf.Session()

                new_saver = tf.train.import_meta_graph(address)
                new_saver.restore(new_sess, tf.train.latest_checkpoint('trained/'))
                training = grapphy_2.get_tensor_by_name('mode:0')
                x = grapphy_2.get_tensor_by_name('x:0')
                softmax_output = grapphy_2.get_tensor_by_name('softmax_output:0')

                res = new_sess.run(softmax_output, feed_dict={x:imgs, training:False})
                results.append(res)
            
        avg_softmax = np.average(results, axis=0)
        my_predicts = np.argmax(a, axis=1)
        
        
        labels = np.argmax(self.labels, axis=1)
        
        
        _, _, accuracy = model[1]
        x, y_, _, _, training, keep_prob = model[0]
        _, _, print_loss = model[3]
        softmax, predicts, test_acc = model[4]
        
        loss_history.append(sess.run(print_loss, feed_dict={softmax: avg_softmax, y_: self.labels, training: False, keep_prob: 1.0}))

        accs.append(sess.run(accuracy, feed_dict={predicts: my_predicts, y_: self.labels, training: False, keep_prob: 1.0}))
        return self

    
#     @action(model='convy')
#     def update_stats(self, model, sess, accs, loss_history):
#         """ Append accuracy that is obtained by convy-model given weights stored in sess Tf-session

#         Args:
#             model: do not supply this arg, always the output of convy-model defined above
#             sess: tf-session with trained (to some extent) weights
#             accs: list with accuracies
#         """
        
        
#         _, _, accuracy = model[1]
#         x, y_, _, _, training, keep_prob = model[0]
#         _, _, print_loss = model[3]
#         loss_history.append(sess.run(print_loss, feed_dict={x: self.images, y_: self.labels, training: False, keep_prob: 1.0}))

#         accs.append(sess.run(accuracy, feed_dict={x: self.images, y_: self.labels, training: False, keep_prob: 1.0}))
#         return self


def draw_stats(stats, title):
    plt.title(title)
    plt.plot(stats)
    plt.xlabel('iteration')
    plt.ylabel('accuracy')
    plt.show()

In [8]:
%matplotlib inline
%env CUDA_VISIBLE_DEVICES = 1

src = 'C:/Users/Dari/Documents/az_training/task_03'

with open(os.path.join(src, 'mnist_pics.blk'), 'rb') as file:
    full_imgs = blosc.unpack_array(file.read())
    
with open(os.path.join(src, 'mnist_labels.blk'), 'rb') as file:
    full_labs = blosc.unpack_array(file.read())
    
src = (np.reshape(full_imgs, (65000, 28, 28)), full_labs)

LEN_MNIST = 65000
indy = DatasetIndex(np.arange(LEN_MNIST))

mnistset = Dataset(indy, batch_class=MnistBatch)
mnistset.cv_split([0.9, 0.1])

env: CUDA_VISIBLE_DEVICES=1


In [9]:
imgs, lbls = src
imgs = imgs[:100, :, :]
lbls = lbls[:100, :]

In [10]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
stats = []
loss_history = []

alpha = 0.001
period = 200
n_iterations = 401

train_loss_history = [] 
train_stats = []

In [11]:
# pipeline for train
ppl = (mnistset.train.pipeline().
       load(src=src, fmt='ndarray').
       train_convy(alpha, period, n_iterations, train_stats, train_loss_history))
    

In [13]:
def f():
    ppl = (mnistset.train.pipeline().
        load(src=src, fmt='ndarray').
        train_convy(alpha, period, n_iterations, train_stats, train_loss_history))

    for i in tqdm(range(n_iterations)):
        ppl.next_batch(100, n_epochs=None)


In [14]:
f()

  0%|                                                                                          | 0/401 [00:00<?, ?it/s]


ValueError: Fetch argument <tf.Tensor 'mul_3:0' shape=() dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("mul_3:0", shape=(), dtype=float32) is not an element of this graph.)

In [11]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'conv2d/kernel:0' shape=(7, 7, 1, 4) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/moving_mean:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/moving_variance:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/kernel:0' shape=(5, 5, 4, 16) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/moving_mean:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/moving_variance:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 16, 32) dtype=float32_ref>,
 <tf.Variable

In [37]:
graphs = {}
for i in range(2):
    graphs['model'+ str(i)] =  i


In [38]:
graphs

{'model0': 0, 'model1': 1}

In [ ]:
n_cycles = 2
period = 200
# for i in tqdm(range(n_iterations)):
#     # ppl.next_batch(100, n_epochs=None)
#     #ppl_test.next_batch(100, n_epochs=None)

results = []
xs = []
sos = []

grapphy_2 = tf.Graph()
graphs = {}
for i in range(1, n_cycles + 1):
    graphs['model'+ str(i)] = tf.Graph()

    with graphs['model'+ str(i)].as_default():
        new_sess = tf.Session()
        
    
    for i in range(1, n_cycles + 1):
    #         with tf.variable_scope(str(n_cycles)):
        folder = 'trained+' + str(i*period) + '/'
        address = folder + 'model' + '-' + str(i*period) + '.meta'
        print ('current', address)
    #                 new_saver = tf.train.import_meta_graph(address)
    #                 new_saver.restore(new_sess, tf.train.latest_checkpoint('trained/'))

    #     grapphy_2 = tf.Graph()
    #     with grapphy_2.as_default():
    # #                 new_sess = tf.Session(graph=grapphy_2)
    #         new_sess = tf.Session()
        
        
        new_saver = tf.train.import_meta_graph(address)
        new_saver.restore(new_sess, tf.train.latest_checkpoint(folder))
        with tf.variable_scope(str(i)):
            training = grapphy_2.get_tensor_by_name('mode:0')
            x = grapphy_2.get_tensor_by_name('x:0')
            softmax_output = grapphy_2.get_tensor_by_name('softmax_output:0')
            xs.append(x)
            sos.append(softmax_output)
        
        #res = new_sess.run(softmax_output, feed_dict={x:imgs, training:False})
        # results.append(res)
        # print (res.shape)

print ('done')

In [14]:
n_cycles = 2
period = 200
# for i in tqdm(range(n_iterations)):
#     # ppl.next_batch(100, n_epochs=None)
#     #ppl_test.next_batch(100, n_epochs=None)

results = []
xs = []
sos = []

grapphy_2 = tf.Graph()

new_sess = tf.Session()

with grapphy_2.as_default():
    new_sess = tf.Session()

    
    for i in range(1, n_cycles + 1):
    #         with tf.variable_scope(str(n_cycles)):
        folder = 'trained+' + str(i*period) + '/'
        address = folder + 'model' + '-' + str(i*period) + '.meta'
        print ('current', address)
    #                 new_saver = tf.train.import_meta_graph(address)
    #                 new_saver.restore(new_sess, tf.train.latest_checkpoint('trained/'))

    #     grapphy_2 = tf.Graph()
    #     with grapphy_2.as_default():
    # #                 new_sess = tf.Session(graph=grapphy_2)
    #         new_sess = tf.Session()
        
        
        new_saver = tf.train.import_meta_graph(address)
        new_saver.restore(new_sess, tf.train.latest_checkpoint(folder))
        with tf.variable_scope(str(i)):
            training = grapphy_2.get_tensor_by_name('mode:0')
            x = grapphy_2.get_tensor_by_name('x:0')
            softmax_output = grapphy_2.get_tensor_by_name('softmax_output:0')
            xs.append(x)
            sos.append(softmax_output)
        
        #res = new_sess.run(softmax_output, feed_dict={x:imgs, training:False})
        # results.append(res)
        # print (res.shape)

print ('done')

current trained+200/model-200.meta


error: invalid group reference 11 at position 1

In [24]:
results = []
for i in range(2):
    res = new_sess.run(sos[i], feed_dict={xs[i]:imgs, training:False})
    results.append(res)

In [13]:
len(grapphy_2.get_operations())

14588

In [9]:
n_cycles = 2
period = 200
# for i in tqdm(range(n_iterations)):
#     # ppl.next_batch(100, n_epochs=None)
#     #ppl_test.next_batch(100, n_epochs=None)

results_2 = []
for i in range(1, n_cycles + 1):
#         with tf.variable_scope(str(n_cycles)):
    address = 'trained/model' + '-' + str(i*period) + '.meta'
#                 new_saver = tf.train.import_meta_graph(address)
#                 new_saver.restore(new_sess, tf.train.latest_checkpoint('trained/'))

    #address = 'trained/model-200.meta'
    grapphy_2 = tf.Graph()
    with grapphy_2.as_default():
#                 new_sess = tf.Session(graph=grapphy_2)
        new_sess = tf.Session()

        new_saver = tf.train.import_meta_graph(address)
        new_saver.restore(new_sess, tf.train.latest_checkpoint('trained/'))
        training = grapphy_2.get_tensor_by_name('mode:0')
        x = grapphy_2.get_tensor_by_name('x:0')
        softmax_output = grapphy_2.get_tensor_by_name('softmax_output:0')
        # new_sess.run(tf.global_variables_initializer())

        res = new_sess.run(softmax_output, feed_dict={x:imgs, training:False})
        results_2.append(res)
        print (res.shape)
print ('done')

INFO:tensorflow:Restoring parameters from trained/model-800
(100, 10)
INFO:tensorflow:Restoring parameters from trained/model-800
(100, 10)
done


In [29]:
results[1] - results[0]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [27]:
results[0]

array([[  1.76795414e-08,   6.81321263e-01,   3.06836754e-01,
          8.99909555e-14,   1.17666749e-02,   2.47535610e-08,
          6.57234050e-05,   2.13701323e-09,   9.40783320e-06,
          1.26009141e-07],
       [  9.87879139e-06,   6.08795047e-01,   4.42498066e-02,
          2.24613488e-01,   4.54826327e-03,   4.45917994e-03,
          1.54893333e-03,   6.35941476e-02,   4.71648276e-02,
          1.01629691e-03],
       [  3.13311205e-07,   1.84587859e-10,   1.43421719e-13,
          1.83362367e-19,   9.99999642e-01,   1.61236004e-15,
          3.55248062e-08,   1.26390705e-28,   4.41992409e-21,
          4.10349380e-14],
       [  4.07662071e-10,   4.44246171e-27,   2.07320393e-23,
          2.49694592e-25,   7.86778559e-26,   1.55530946e-35,
          1.24471696e-34,   1.06452911e-11,   2.60874939e-28,
          1.00000000e+00],
       [  2.21930793e-03,   9.92611229e-01,   6.83569760e-06,
          2.93956390e-07,   1.70885539e-03,   2.09651282e-16,
          3.31501896e-03